In [65]:
import pandas as pd
from path import Path
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [66]:
diamond_df = pd.read_csv(Path("Diamonds_price_data.csv"))
diamond_df.head

<bound method NDFrame.head of        Unnamed: 0  carat        cut color clarity  depth  table  price     x  \
0               1   0.23      Ideal     E     SI2   61.5   55.0    326  3.95   
1               2   0.21    Premium     E     SI1   59.8   61.0    326  3.89   
2               3   0.23       Good     E     VS1   56.9   65.0    327  4.05   
3               4   0.29    Premium     I     VS2   62.4   58.0    334  4.20   
4               5   0.31       Good     J     SI2   63.3   58.0    335  4.34   
...           ...    ...        ...   ...     ...    ...    ...    ...   ...   
53938       53939   0.86    Premium     H     SI2   61.0   58.0   2757  6.15   
53939       53940   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83   
53940       53941   0.71    Premium     E     SI1   60.5   55.0   2756  5.79   
53941       53942   0.71    Premium     F     SI1   59.8   62.0   2756  5.74   
53942       53943   0.70  Very Good     E     VS2   60.5   59.0   2757  5.71   

         

In [67]:
diamond_df['color'] = diamond_df['color'].replace(['J','I','H','G','F','E','D'],[1,2,3,4,5,6,7])
diamond_df['cut'] = diamond_df['cut'].replace(['Fair','Good','Very Good','Premium','Ideal'],[1,2,3,4,5])
diamond_df['clarity'] = diamond_df['clarity'].replace(['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF'],[1,2,3,4,5,6,7,8,9,10])

features = diamond_df.drop(columns=["price"])

diamond_df.head()



,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,5,6,4,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,4,6,5,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,2,6,7,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,4,2,6,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,2,1,4,63.3,58.0,335,4.34,4.35,2.75


In [68]:
# labels is the value we want to predict (price).
labels = np.array(diamond_df['price'])

# saving feature names for later use.
feature_list = list(features.columns)

# converting to numpy array.
features = np.array(features)

In [69]:
# splitting data into training and testing sets.
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

# instantiate model.
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# fit the model to the data.
rf.fit(train_features, train_labels);

KeyboardInterrupt: 

In [ ]:
predictions = rf.predict(test_features)

# calculate the absolute errors
errors = abs(predictions - test_labels)

# print out the mean absolute error.
print('Mean Absolute Error:', round(np.mean(errors), 2), 'dollars')

# calculate mean absolute percentage error.
mape = 100 * (errors / test_labels)

# calculate accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# numerical feature importances.
importances = list(rf.feature_importances_)

# list of tuples w/ features matched with their improtance.
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# sort most important features first.
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# print out feature and corresponding importances.
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];
